In [24]:
# Importing required libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import csv

# Kör utan GUI
options = Options()
#options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")


# Initialization of variables
base_url = "https://www.blocket.se"
pagination_url = "https://www.blocket.se/bilar/sok?filter=%7B%22key%22%3A%22price%22%2C%22range%22%3A%7B%22start%22%3A%2220000%22%2C%22end%22%3A%22%22%7D%7D&filter=%7B%22key%22%3A%22modelYear%22%2C%22range%22%3A%7B%22start%22%3A%222005%22%2C%22end%22%3A%22%22%7D%7D&filter=%7B%22key%22%3A%22ownershipType%22%2C%22values%22%3A%5B%22K%C3%B6pa%22%5D%7D&filter=%7B%22key%22%3A%22fuel%22%2C%22values%22%3A%5B%22El%22%5D%7D&q=EJ+a-traktor&page="
total_no_of_pages = 1 # Verkligt antal sidor
# total_no_of_pages = 2 # För test

link_list = []

# Start Chrome WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Function to get BeautifulSoup DOM from a given URL
def get_dom(url):
    driver.get(url)
    time.sleep(3)  # Wait for JS to load
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup

# Loop through pages and collect URLs
for page_no in range(1, total_no_of_pages + 1):
    page_url = pagination_url + str(page_no)
    soup = get_dom(page_url)
    
    # Find the main container with class="list w-full"
    container = soup.find("div", class_="list w-full")
    
    if container:
        links = container.find_all("a", href=True)
        for a in links:
            full_url = a["href"]
            if not full_url.startswith("http"):
                full_url = base_url + full_url
            link_list.append(full_url)

# Output result
print("Antal länkar hittade:", len(link_list))
#for link in link_list:
#    print(link)
with open('lankar.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['URL'])  # Rubrikrad
    for link in link_list:
        csv_writer.writerow([link])

# Stäng WebDriver
driver.quit()


Antal länkar hittade: 40


In [28]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

resultat_lista = []
link_list2 = ["https://www.blocket.se/annons/1002069529"]

for link in link_list2:
    try:
        response = requests.get(link, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            element = soup.find(id="skip-tabbar")

            if element:
                text = element.get_text()
                match = re.search(r"eMJjcF(\w+)", text)
                if match:
                    resultat = match.group(1)
                    resultat_lista.append((link, resultat))
                else:
                    resultat_lista.append((link, "Ingen match"))
            else:
                resultat_lista.append((link, "Element saknas"))
        else:
            resultat_lista.append((link, f"Felkod {response.status_code}"))

        # Vänta lite mellan förfrågningar
        time.sleep(1 + random.uniform(0, 1.5))

    except Exception as e:
        resultat_lista.append((link, f"Fel: {str(e)}"))

# Spara till CSV
with open('resultat.csv', 'w', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Länk', 'Matchad sträng'])
    for row in resultat_lista:
        csv_writer.writerow(row)


In [38]:
import requests
import csv
import re
import time
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

resultat_lista = []

for link in link_list2:
    try:
        response = requests.get(link, headers=headers, timeout=10)
        if response.status_code == 200:
            html = response.text

            # Hitta alla förekomster av "</div>" och extrahera texten innan
            matches = re.findall(r'>([^<]+)</div>', html)

            if matches:
                for match in matches:
                    resultat_lista.append((link, match.strip()))
            else:
                resultat_lista.append((link, "Ingen match"))

        else:
            resultat_lista.append((link, f"Felkod {response.status_code}"))

        # Vänta lite mellan förfrågningar
        time.sleep(1 + random.uniform(0, 1.5))

    except Exception as e:
        resultat_lista.append((link, f"Fel: {str(e)}"))

# Spara resultat till CSV
with open('div_innehall.csv', 'w', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Länk', 'Innehåll före </div>'])

    for row in resultat_lista:
        csv_writer.writerow(row)

print(resultat_lista)

[('https://www.blocket.se/annons/1002069529', 'Blocket är en del av Schibsted Marketplaces.')]


In [42]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

resultat_lista = []

for link in link_list2:
    try:
        response = requests.get(link, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Hämta alla <div> där någon av klasserna är 'eMJjcF'
            #divs = soup.find_all("div", class_="eMJjcF")
            divs = soup.find_all("div", class_=lambda x: x and "eMJjcF" in x)


            if divs:
                for div in divs:
                    text = div.get_text(strip=True)
                    resultat_lista.append((link, text))
            else:
                resultat_lista.append((link, "Ingen match"))

        else:
            resultat_lista.append((link, f"Felkod {response.status_code}"))

        time.sleep(1 + random.uniform(0, 1.5))

    except Exception as e:
        resultat_lista.append((link, f"Fel: {str(e)}"))

# Spara till CSV
with open('matchade_divar.csv', 'w', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Länk', 'Text i <div class="eMJjcF">'])
    for row in resultat_lista:
        csv_writer.writerow(row)
        
print(resultat_lista)

[('https://www.blocket.se/annons/1002069529', 'Ingen match')]


In [ ]:
import re

resultat_lista = []

for link in link_list:
    try:
        driver.get(link)
        time.sleep(5)  # Vänta på att sidan ska ladda

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        element = soup.find(id="skip-tabbar")

        if element:
            text = element.get_text()
            match = re.search(r"eMJjcF(\w+)", text)
            if match:
                resultat = match.group(1)
                resultat_lista.append((link, resultat))
            else:
                resultat_lista.append((link, "Ingen match"))
        else:
            resultat_lista.append((link, "Element saknas"))

    except Exception as e:
        resultat_lista.append((link, f"Fel: {str(e)}"))

# Spara i CSV
with open('resultat.csv', 'w', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['Länk', 'Matchad sträng'])
    for row in resultat_lista:
        csv_writer.writerow(row)